In [1]:
# import libraries
import pandas as pd
import numpy as np
import datetime

In [2]:
# display max columns to None to see everything and account for potential additions
pd.set_option('display.max_columns', None)

# load data for passing
pass_to_df = pd.read_excel('../Data/NFL_Passing_Data.xlsx', sheet_name = None)
rush_to_df = pd.read_excel('../Data/NFL_Rushing_Data.xlsx', sheet_name = None)

# concat all sheets into dataframe
nfl_pass = pd.concat(pass_to_df, axis = 0, ignore_index = True)
nfl_rush = pd.concat(rush_to_df, axis = 0, ignore_index = True)

# quick inspection
nfl_pass.head()

,Rk,Tm,Year,Date,Time,LTime,Unnamed: 6,Opp,Week,G#,Day,Result,OT,Cmp,Att,Cmp%,Yds,TD,Int,Sk,Yds.1,Rate
0,43,ARI,2010,2010-09-12,04:15:00,03:15:00,@,STL,1,1,Sun,W 17-13,NaN,22,41,53.7,266,1,0,2,31,85.1
1,107,ARI,2010,2010-09-19,01:03:00,01:03:00,@,ATL,2,2,Sun,L 7-41,NaN,18,33,54.5,149,0,3,2,15,30.4
2,152,ARI,2010,2010-09-26,04:15:00,01:15:00,NaN,OAK,3,3,Sun,W 24-23,NaN,12,26,46.2,108,2,1,2,14,69.7
3,140,ARI,2010,2010-10-03,04:15:00,01:15:00,@,SDG,4,4,Sun,L 10-41,NaN,15,28,53.6,87,0,2,9,59,38.7
4,135,ARI,2010,2010-10-10,04:05:00,01:05:00,NaN,NOR,5,5,Sun,W 30-20,NaN,18,29,62.1,153,0,1,4,21,64.4


In [3]:
# quick inspection
nfl_rush.head()

,Rk,Tm,Year,Date,Time,LTime,Unnamed: 6,Opp,Week,G#,Day,Result,OT,Att,Yds,Y/A,TD
0,116,ARI,2010,2010-09-12,04:15:00,03:15:00,@,STL,1,1,Sun,W 17-13,NaN,21,112,5.33,1
1,166,ARI,2010,2010-09-19,01:03:00,01:03:00,@,ATL,2,2,Sun,L 7-41,NaN,13,118,9.08,1
2,68,ARI,2010,2010-09-26,04:15:00,01:15:00,NaN,OAK,3,3,Sun,W 24-23,NaN,26,119,4.58,0
3,162,ARI,2010,2010-10-03,04:15:00,01:15:00,@,SDG,4,4,Sun,L 10-41,NaN,14,47,3.36,0
4,86,ARI,2010,2010-10-10,04:05:00,01:05:00,NaN,NOR,5,5,Sun,W 30-20,NaN,24,41,1.71,0


In [4]:
# Focus on passing first
# list all columns
nfl_pass.columns

Index(['Rk', 'Tm', 'Year', 'Date', 'Time', 'LTime', 'Unnamed: 6', 'Opp',
       'Week', 'G#', 'Day', 'Result', 'OT', 'Cmp', 'Att', 'Cmp%', 'Yds', 'TD',
       'Int', 'Sk', 'Yds.1', 'Rate'],
      dtype='object')

In [5]:
# drop unnessary columns
nfl_pass = nfl_pass.drop(['Rk', 'Time', 'LTime', 'Unnamed: 6', 'Opp', 'Result', 'OT', 'Yds.1'], axis = 1)

# make column headers uniform
nfl_pass.columns = nfl_pass.columns.map(lambda x: x.replace(' ', '_').lower())

# rename columns
nfl_pass.rename(columns = {'tm': 'team', 'year': 'season', 'g#': 'game', 'cmp': 'team_cmp', 'att': 'team_p_att',
                           'cmp%': 'team_cmp%', 'yds': 'team_p_yards', 'td': 'team_p_td', 'sk': 'team_sacked',
                           'rate': 'team_qbr'}, inplace = True)

# replace cmp% value with calculated value
nfl_pass['team_cmp%'] = round((nfl_pass.team_cmp / nfl_pass.team_p_att),3)

# convert days
day = {'day': {'Sun': 0, 'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6}}
nfl_pass.replace(day, inplace = True)

# inspect
nfl_pass.head()

,team,season,date,week,game,day,team_cmp,team_p_att,team_cmp%,team_p_yards,team_p_td,int,team_sacked,team_qbr
0,ARI,2010,2010-09-12,1,1,0,22,41,0.537,266,1,0,2,85.1
1,ARI,2010,2010-09-19,2,2,0,18,33,0.545,149,0,3,2,30.4
2,ARI,2010,2010-09-26,3,3,0,12,26,0.462,108,2,1,2,69.7
3,ARI,2010,2010-10-03,4,4,0,15,28,0.536,87,0,2,9,38.7
4,ARI,2010,2010-10-10,5,5,0,18,29,0.621,153,0,1,4,64.4


In [6]:
# Focus on rushing
# list all columns
nfl_rush.columns

Index(['Rk', 'Tm', 'Year', 'Date', 'Time', 'LTime', 'Unnamed: 6', 'Opp',
       'Week', 'G#', 'Day', 'Result', 'OT', 'Att', 'Yds', 'Y/A', 'TD'],
      dtype='object')

In [7]:
# drop unnessary columns
nfl_rush = nfl_rush.drop(['Rk', 'Time', 'LTime', 'Unnamed: 6', 'Opp', 'Result', 'OT'], axis = 1)

# make column headers uniform
nfl_rush.columns = nfl_rush.columns.map(lambda x: x.replace(' ', '_').lower())

# rename columns
nfl_rush.rename(columns = {'tm': 'team', 'year': 'season', 'g#': 'game', 'att': 'team_r_att', 'yds': 'team_r_yards', 
                           'y/a': 'team_r_yrd_att', 'td': 'team_r_td'}, inplace = True)

# replace team_r_yrd_att value with calculated value
nfl_rush['team_r_yrd_att'] = round((nfl_rush.team_r_yards / nfl_rush.team_r_att),3)

# convert days
day = {'day': {'Sun': 0, 'Mon': 1, 'Tue': 2, 'Wed': 3, 'Thu': 4, 'Fri': 5, 'Sat': 6}}
nfl_rush.replace(day, inplace = True)

# inspect
nfl_rush.head()

,team,season,date,week,game,day,team_r_att,team_r_yards,team_r_yrd_att,team_r_td
0,ARI,2010,2010-09-12,1,1,0,21,112,5.333,1
1,ARI,2010,2010-09-19,2,2,0,13,118,9.077,1
2,ARI,2010,2010-09-26,3,3,0,26,119,4.577,0
3,ARI,2010,2010-10-03,4,4,0,14,47,3.357,0
4,ARI,2010,2010-10-10,5,5,0,24,41,1.708,0


In [8]:
# merge dataframes
nfl_stats = pd.merge(nfl_pass, nfl_rush, how = 'left', on = ['team', 'season', 'date', 'week', 'game', 'day'])
nfl_stats.head()

,team,season,date,week,game,day,team_cmp,team_p_att,team_cmp%,team_p_yards,team_p_td,int,team_sacked,team_qbr,team_r_att,team_r_yards,team_r_yrd_att,team_r_td
0,ARI,2010,2010-09-12,1,1,0,22,41,0.537,266,1,0,2,85.1,21,112,5.333,1
1,ARI,2010,2010-09-19,2,2,0,18,33,0.545,149,0,3,2,30.4,13,118,9.077,1
2,ARI,2010,2010-09-26,3,3,0,12,26,0.462,108,2,1,2,69.7,26,119,4.577,0
3,ARI,2010,2010-10-03,4,4,0,15,28,0.536,87,0,2,9,38.7,14,47,3.357,0
4,ARI,2010,2010-10-10,5,5,0,18,29,0.621,153,0,1,4,64.4,24,41,1.708,0


In [9]:
nfl_stats.team.unique()

array(['ARI', 'ATL', 'BAL', 'BUF', 'CAR', 'CHI', 'CIN', 'CLE', 'DAL',
       'DEN', 'DET', 'GNB', 'HOU', 'IND', 'JAX', 'KAN', 'SDG', 'LAC',
       'STL', 'LAR', 'OAK', 'LVR', 'MIA', 'MIN', 'NWE', 'NOR', 'NYG',
       'NYJ', 'PHI', 'PIT', 'SEA', 'SFO', 'TAM', 'TEN', 'WAS'],
      dtype=object)

In [10]:
nfl_stats.replace({'team': {'ARI': 'Arizona Cardinals', 'ATL': 'Atlanta Falcons', 'BAL': 'Baltimore Ravens',
                            'BUF': 'Buffalo Bills', 'CAR': 'Carolina Panthers', 'CHI': 'Chicago Bears',
                            'CIN': 'Cincinnati Bengals', 'CLE': 'Cleveland Browns', 'DAL': 'Dallas Cowboys',
                            'DEN': 'Denver Broncos', 'DET': 'Detroit Lions', 'GNB': 'Green Bay Packers',
                            'HOU': 'Houston Texans', 'IND': 'Indianapolis Colts', 'JAX': 'Jacksonville Jaguars',
                            'KAN': 'Kansas City Chiefs', 'OAK': 'Las Vegas Raiders', 'LVR': 'Las Vegas Raiders',
                            'SDG': 'Los Angeles Chargers', 'LAC': 'Los Angeles Chargers', 'STL': 'Los Angeles Rams',
                            'LAR': 'Los Angeles Rams', 'MIA': 'Miami Dolphins', 'MIN': 'Minnesota Vikings',
                            'NWE': 'New England Patriots', 'NOR': 'New Orleans Saints', 'NYG': 'New York Giants',
                            'NYJ': 'New York Jets', 'PHI': 'Philadelphia Eagles', 'PIT': 'Pittsburgh Steelers',
                            'SFO': 'San Francisco 49ers', 'SEA': 'Seattle Seahawks', 'TAM': 'Tampa Bay Buccaneers',
                            'TEN': 'Tennessee Titans', 'WAS': 'Washington Football Team'}}, inplace = True)

In [11]:
nfl_stats.head()

,team,season,date,week,game,day,team_cmp,team_p_att,team_cmp%,team_p_yards,team_p_td,int,team_sacked,team_qbr,team_r_att,team_r_yards,team_r_yrd_att,team_r_td
0,Arizona Cardinals,2010,2010-09-12,1,1,0,22,41,0.537,266,1,0,2,85.1,21,112,5.333,1
1,Arizona Cardinals,2010,2010-09-19,2,2,0,18,33,0.545,149,0,3,2,30.4,13,118,9.077,1
2,Arizona Cardinals,2010,2010-09-26,3,3,0,12,26,0.462,108,2,1,2,69.7,26,119,4.577,0
3,Arizona Cardinals,2010,2010-10-03,4,4,0,15,28,0.536,87,0,2,9,38.7,14,47,3.357,0
4,Arizona Cardinals,2010,2010-10-10,5,5,0,18,29,0.621,153,0,1,4,64.4,24,41,1.708,0


In [12]:
nfl_stats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5632 entries, 0 to 5631
Data columns (total 18 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   team            5632 non-null   object        
 1   season          5632 non-null   int64         
 2   date            5632 non-null   datetime64[ns]
 3   week            5632 non-null   int64         
 4   game            5632 non-null   int64         
 5   day             5632 non-null   int64         
 6   team_cmp        5632 non-null   int64         
 7   team_p_att      5632 non-null   int64         
 8   team_cmp%       5632 non-null   float64       
 9   team_p_yards    5632 non-null   int64         
 10  team_p_td       5632 non-null   int64         
 11  int             5632 non-null   int64         
 12  team_sacked     5632 non-null   int64         
 13  team_qbr        5632 non-null   float64       
 14  team_r_att      5632 non-null   int64         
 15  team

In [13]:
# save cleaned data
nfl_stats.to_csv('..\\Data\\NFL_Run_Pass_Data_Cleaned.csv', index = False) 
print('nfl_stats has been saved as CSV file')

nfl_stats has been saved as CSV file
